In [296]:
"""
1. Information extraction from PDF documents.
"""
# importing required packages
import re
import nltk
import json
from pathlib import Path
from tika import parser

# fetching_path_to_documents_location_from_configuration_file
with open("conf.json") as json_conf : 
    CONF = json.load(json_conf)

directory = format(CONF["base_path"])
#print(format(CONF["base_path"]))
    
# function to extract grammer pattern from text
def grammerPatternExtraction(chunkName,attribute_list):
    # for each phrase sub tree in the parse tree
    for subtree in tree.subtrees(filter=lambda t: t.label() == chunkName):
        # appending the phrase from a list of part-of-speech tagged words
        attribute_list.append(" ".join([word for word, pos in subtree]))
    return attribute_list

# traversing through all the docs in the directory
for filename in os.listdir(directory):
    # filtering the docs ending with .pdf's and .PDF's
    if filename.endswith(".pdf") or filename.endswith(".PDF"): 
        # building the path to each file
        file = Path(directory) / filename
        print(filename)
        print(file)
        print("===============================================================")
        # using tika's parser to extract text from pdf
        text = parser.from_file(str(file))
        # extracting required content from tika's output
        text=text["content"]
        text = text.replace("\n","")
        #print(text)
        # extracting the part of text lying between "Rounding" and "Valuation and Timing" keywords
        result = re.search("%s(.*)%s" % ("Rounding", "Valuation and Timing"), text).group(1)
        # sentences tokenizer
        sentences = nltk.sent_tokenize(str(result))
        # word tokenize each sentence
        sentences = [nltk.word_tokenize(sent) for sent in sentences]
        # pos tagging each sentence
        sentences = [nltk.pos_tag(sent) for sent in sentences]
        #print(sentences)
        # defining a grammar with a regular-expression rule 
        grammar = r"""
        NP: {<DT><NNP><NNP>} 
        VP: {<MD><VB><VBN>} 
        PP: {<R.*>}
        ND: {<NNP><CD>}
            {<NNP><,><NNP>}
        """
        # creating a chunk parser
        cp = nltk.RegexpParser(grammar)
        # looping over all the sentences from extracted text
        for sent in sentences:
            tree = cp.parse(sent)
            amount_type=[]
            relation=[]
            indicator=[]
            amount=[]
            currancy_pattern = "USD|INR|EUR|GBP"
            # calling grammerPatternExtraction method to extract the words which has required chunk grammer patterns
            amount_type = grammerPatternExtraction("NP",amount_type)
            relation = grammerPatternExtraction("VP",relation)
            indicator = grammerPatternExtraction("PP",indicator)
            amount = grammerPatternExtraction("ND",amount)
            #extracting required keywords and relations from list of extracted words in grammerPatternExtraction
            if len(amount_type)>0:
                print(amount_type[0])
            # skipping the sentences which does't have required grammer patterns
            else:
                continue
            if len(relation)>0 and len(indicator)>0:
                print(relation[0]+" : "+indicator[0])
            # skipping the sentences which does't have required grammer patterns
            else:
                continue
            if len(amount)>0:
                currancy = "".join(re.findall(currancy_pattern,amount[0]))
                print("currancy : "+currancy)
                print("amount :"+amount[0].replace(currancy,""))
            # skipping the sentences which does't have required grammer patterns
            else:
                continue
            print("---------------------------")
            if len(amount_type)>1:
                print(amount_type[1])
            if len(relation)>0:
                if len(indicator)>1:
                    print(relation[0]+" : "+indicator[1])
                elif len(indicator)>0:
                    print(indicator[0]+" : "+indicator[0])
            # separating currancy and amount from extracted amount
            if len(amount)>1:
                currancy = "".join(re.findall(currancy_pattern,amount[1]))
                print("currancy : "+currancy)
                print("amount :"+amount[1].replace(currancy,""))
            elif len(amount)>0:
                currancy = "".join(re.findall(currancy_pattern,amount[0]))
                print("currancy : "+currancy)
                print("amount :"+amount[0].replace(currancy,""))
            #print(tree)
            #result.draw()
            print("---------------------------")
        print()

16017sec.pdf
C:\Users\mchai\Downloads\stride\16017sec.pdf
The Delivery Amount
will be rounded : up
currancy : USD
amount : 10,000
---------------------------
the Return Amount
will be rounded : down
currancy : USD
amount : 10,000
---------------------------

20197sec.pdf
C:\Users\mchai\Downloads\stride\20197sec.pdf
The Delivery Amount
will be rounded : up
currancy : EUR
amount : 10,000
---------------------------
the Return Amount
will be rounded : down
currancy : EUR
amount : 10,000
---------------------------

English law VM CSA EXECUTED.PDF
C:\Users\mchai\Downloads\stride\English law VM CSA EXECUTED.PDF
the Delivery Amount
will be rounded : up
currancy : USD
amount : 10,000
---------------------------
the Return Amount
will be rounded : down
currancy : USD
amount : 10,000
---------------------------

HBUS00002134-00002150.pdf
C:\Users\mchai\Downloads\stride\HBUS00002134-00002150.pdf
The Delivery Amount
shall be rounded : up
currancy : USD
amount :lOO , OOO
--------------------------

In [294]:
"""
2. Identify noun phrases from given input text.
"""
# importing required packages
import os
import re
import nltk
from tika import parser

# fetching_path_to_documents_location_from_configuration_file
with open("conf.json") as json_conf : 
    CONF = json.load(json_conf)

directory = format(CONF["base_path"])
#print(format(CONF["base_path"]))

# function to extract grammer pattern from text
def grammerPatternExtraction(chunkName,attribute_list):
    # for each phrase sub tree in the parse tree
    for subtree in tree.subtrees(filter=lambda t: t.label() == chunkName):
        # appending the phrase from a list of part-of-speech tagged words
        attribute_list.append(" ".join([word for word, pos in subtree]))
    return attribute_list

# traversing through all the docs in the directory
for filename in os.listdir(directory):
    # filtering the docs ending with .pdf's and .PDF's
    if filename.endswith(".pdf") or filename.endswith(".PDF"): 
        # building the path to each file
        file = os.path.join(directory, filename)
        print(filename)
        print("===============================================================")
        # using tika's parser to extract text from pdf
        text = parser.from_file(file)
        # extracting required content from tika's output
        text=text["content"]
        text = text.replace("\n","")
        #print(text)
        # sentences tokenizer
        sentences = nltk.sent_tokenize(str(text))
        # word tokenize each sentence
        sentences = [nltk.word_tokenize(sent) for sent in sentences]
        # pos tagging each sentence
        sentences = [nltk.pos_tag(sent) for sent in sentences]
        #print(sentences)
        # defining a grammar with a regular-expression rule 
        grammar = r"""
        NP: {<NN.*>}
        """
        # creating a chunk parser
        cp = nltk.RegexpParser(grammar)
        # looping over all the sentences from extracted text
        noun_phrases = []
        for sent in sentences:
            tree = cp.parse(sent)
            # calling grammerPatternExtraction method to extract the words which has required chunk grammer patterns
            noun_phrases = grammerPatternExtraction("NP",noun_phrases)
        # filterng list elements with character length one
        noun_phrases = [word for word in noun_phrases if len(word)>1]
        # filtering alphanumeric and numeric elements from list
        noun_phrases = [word for word in noun_phrases if word.isalpha()]
        # printing the length of list
        print("No. of word phrases in the documnet : "+str(len(list(set(noun_phrases)))))
        # printing the final list of noun phrases from documents
        print("Noun phrases in the document : ")
        print(list(set(noun_phrases)))
        print("---------------------------")

16017sec.pdf
No. of word phrases in the documnet : 393
Noun phrases in the document : 
['ExchangeDate', 'Time', 'market', 'DefaultThe', 'demand', 'avoidance', 'attempt', 'deposits', 'dispute', 'source', 'number', 'Exposure', 'Good', 'advisers', 'transfers', 'Persons', 'extent', 'replaces', 'Users', 'description', 'Valub', 'minus', 'doubt', 'conditions', 'payment', 'Paragraphs', 'use', 'cither', 'supersedes', 'trade', 'stamp', 'conflict', 'owner', 'Deed', 'property', 'obligations', 'transactions', 'dates', 'Timing', 'Amount', 'agreements', 'appetite', 'Matters', 'communication', 'Title', 'Transfers', 'sufficient', 'charges', 'calculations', 'Subject', 'Representation', 'costs', 'end', 'part', 'Address', 'Daily', 'Market', 'transmission', 'Equivalents', 'Nothing', 'creation', 'ValueThe', 'ANNEXto', 'sum', 'party', 'metals', 'Amounts', 'entirety', 'Bilateral', 'securities', 'ExpensesEach', 'London', 'lien', 'amount', 'business', 'pledge', 'Definition', 'Calculations', 'Elections', 'Agreem

No. of word phrases in the documnet : 447
Noun phrases in the document : 
['Time', 'System', 'data', 'market', 'demand', 'avoidance', 'attempt', 'deposits', 'dispute', 'number', 'Inconsistency', 'Exposure', 'Good', 'advisers', 'transfers', 'Persons', 'option', 'extent', 'Netting', 'Users', 'description', 'doubt', 'conditions', 'payment', 'Paragraphs', 'use', 'Fleet', 'trade', 'thereunder', 'stamp', 'midmarket', 'Reinstatement', 'owner', 'collection', 'Deed', 'obligations', 'property', 'transactions', 'Values', 'Timing', 'Amount', 'Matters', 'Compounding', 'communication', 'publication', 'None', 'Title', 'books', 'Expenses', 'Transfers', 'sufficient', 'charges', 'calculations', 'information', 'Subject', 'Representation', 'costs', 'end', 'part', 'Market', 'Payer', 'Equivalents', 'course', 'Nothing', 'foregoing', 'creation', 'Legal', 'Conditions', 'sum', 'party', 'vii', 'entirety', 'Bilateral', 'Excluded', 'securities', 'Offsets', 'Haircut', 'nothing', 'London', 'lien', 'amount', 'busines

In [284]:
"""
1. Information extraction from PDF documents.
"""
# importing required packages
import json

# path_to_document_location
#directory = "C:/Users/mchai/Downloads/stride/"

with open(r"conf.json") as json_conf : 
    CONF = json.load(json_conf)
print(format(CONF["base_path"]))

C:/Users/mchai/Downloads/stride/
